# CheckThat 2022 Baseline

*** Author: Mina Schütz ***
*** E-Mail: mina.schuetz@h-da.de (1) mina.schuetz@ait.ac.at (2) ***
*** Affiliation: Darmstadt University for Applied Sciences (1) and Austrian Institute of Technology GmbH (2) ***

In [9]:
import transformers
# from transformers import *
import tensorflow as tf
import sklearn
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import torch
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow import keras
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
from itertools import chain

In [2]:
def load_data(file_path):
    '''
        Params:
        --file_path (Str) = Path to TSV/CSV file
        Retunrs:
        --df (DataFrame) = dataframe with containing the testdataset
    '''

    df = pd.read_csv(file_path, sep="\t", encoding="utf-8", index_col=[0])
    return df

In [3]:
def prepare_data(df):
    '''
        This function changes the classes into numerical values and deletes all NullValues found in the text content
        Params:
         --df (DataFrame) = DataFrame containing the testset
        Returns:
        --df (DataFrame) = Cleaned DataFrame, prepared for preprocessing
    
    '''

    df["our rating"] = np.where(df["our rating"] == "partially false", 2, df["our rating"])
    df["our rating"] = np.where(df["our rating"] == "false", 1, df["our rating"])
    df["our rating"] = np.where(df["our rating"] == "true", 0, df["our rating"])
    df["our rating"] = np.where(df["our rating"] == "other", 3, df["our rating"])
    df['text'].replace('', np.nan, inplace=True)
    df['title'].replace('', np.nan, inplace=True)
    df.dropna(subset=['title'], inplace=True)
    df.dropna(subset=['text'], inplace=True)
    df.dropna(subset = ["our rating"], inplace=True)
    df['text'] = df['title'] +' '+ df['text']

    return df

In [4]:
def load_model(model_path):

    '''
        Params:
        -- model_path = Path to local trained model
        Returns:
        -- model = BERT-model for testing
        -- tokenizer = original BERT tokenizer from HuggingFace
    
    '''

    tokenizer = BertTokenizer.from_pretrained('bert-base-cased', lower_case=False)
    model = BertForSequenceClassification.from_pretrained(
        model_path, 
        num_labels = 4,
        output_attentions = False, 
        output_hidden_states = False, 
    )

    return model, tokenizer


In [5]:
def preprocess_data(df, tokenizer):

    '''
        Preparation of a whole dataset for a forward pass through the model.
        
        Params:
        -- df = Cleaned DataFrame
        -- tokenizer = BERT Tokenizer
        Returns:
        -- Torch DataLoader for the Forward Pass
    
    '''

    MAX_LEN = 512
    batch_size = 6

    test_sentences = df.text.to_list()
    test_labels = df["our rating"].to_list()

    test_ids = []
    
    for sent in test_sentences:
        encoded_sent = tokenizer.encode(sent, add_special_tokens = True)
        test_ids.append(encoded_sent)

    test_ids = pad_sequences(test_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")
    
    test_masks = []
    for sent in test_ids:
        att_mask = [int(token_id > 0) for token_id in sent]
        test_masks.append(att_mask)

    test_inputs = torch.tensor(test_ids)
    test_masks = torch.tensor(test_masks)
    test_labels = torch.tensor(test_labels)

    test_data = TensorDataset(test_inputs, test_masks, test_labels)
    test_sampler = SequentialSampler(test_data)
    test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

    return test_dataloader

In [6]:
def eval_pass(test_dataloader, model, df):

    '''
        Evaluation of the testset.

        Params: 
        -- test_dataloader = Torch DataLoader
        -- model = The loaded model from disk
    
    '''

    print('Prediction started')

    # Set device for PyTorch
    if torch.cuda.is_available():    
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    # Push model to device
    model.to(device)

    # Check if GPU or CPU 
    if torch.cuda.is_available():    
        model.cuda()
    else:
        print('No GPU available, using the CPU instead.')
        model.cpu()
    
    # Model to evaluation mode
    model.eval()

    # Helper variables for eval metrics
    test_accuracy, test_fone, test_precision, test_recall, nb_test_steps = 0, 0, 0, 0, 0
    labels_arr, softmax_arr, logits_arr, argmax_arr = [], [], [], []

    # Go through the batches in the Prediction Dataloader
    for batch in test_dataloader:
        
        #> Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        #> Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            
            #> Forward pass, calculate logit predictions
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # Detach logit outputs and labels from model
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        logits_arr.append(logits)

        # Softmax on logits for final class
        logits_soft_eval = tf.nn.softmax(logits)
        logits_arg_eval = np.argmax(logits_soft_eval, axis=1)
        
        # Calculate eval metrics per batch
        tmp_test_accuracy = accuracy_score(label_ids, logits_arg_eval)
        tmp_test_precision = precision_score(label_ids, logits_arg_eval, average="macro")
        tmp_test_recall = recall_score(label_ids, logits_arg_eval, average="macro")
        tmp_test_fone = f1_score(label_ids, logits_arg_eval, average="macro")
    
        # Add the metrics to a global variable to calculate the actual outcome after the testing
        test_accuracy += tmp_test_accuracy
        test_precision += tmp_test_precision
        test_recall += tmp_test_recall
        test_fone += tmp_test_fone
    
        nb_test_steps += 1   
        
        # Divide the total metric with the testing steps to get the result
        # Accuracy, Precision, Recall, and F1
        
    for line in logits_arr:
        logits_soft_tmp = tf.nn.softmax(line)
        logits_arg_tmp = np.argmax(logits_soft_tmp, axis=1)
        softmax_arr.append(logits_soft_tmp)
        argmax_arr.append(logits_arg_tmp)
    
    argmax_arr = list(chain.from_iterable(argmax_arr)) 
    
    new_list = []
    pred_0, pred_1, pred_2, pred_3 = [], [], [], []
    
    for line in softmax_arr:
        b = line.numpy()
        new_list.append(b)

    for line in new_list:
        for item in line:
            pred_0.append(item[0])
            pred_1.append(item[1])
            pred_2.append(item[2])
            pred_3.append(item[3])

    df["true"] = pred_0
    df["false"] = pred_1
    df["partially false"] = pred_2
    df["other"] = pred_3
    df["final_prediction"] = argmax_arr
    
    print(" ")
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")
    print(" ")
    print("  Accuracy: {0:.2f}".format(test_accuracy/nb_test_steps))
    print(" ")
    print("  Precision: {0:.2f}".format(test_precision/nb_test_steps))
    print(" ")
    print("  Recall: {0:.2f}".format(test_recall/nb_test_steps))
    print(" ")
    print("  F1: {0:.2f}".format(test_fone/nb_test_steps))
    print(" ")
    print('  Testing complete.')
    print(" ")
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")
    
    return df

In [7]:
file_path = "Task3a_testing.tsv"
model_path = "model_save_3/"

In [8]:
df = load_data(file_path)
df = prepare_data(df)
model, tokenizer = load_model(model_path)
test_dataloader = preprocess_data(df, tokenizer)

# Test model
df = eval_pass(test_dataloader, model, df)

Downloading: 100%|██████████████████████████████████████████████████████████████████| 208k/208k [00:00<00:00, 2.94MB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 29.0/29.0 [00:00<00:00, 14.8kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 570/570 [00:00<00:00, 190kB/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (607 > 512). Running this sequence through the model will result in indexing errors


Prediction started


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)

In [9]:
# True = Class 0
# False = Class 1
# Partially False = Class 3
# Other = Class 4
df

,text,title,our rating,true,false,partially false,other,final_prediction
public_id,,,,,,,,
81a67c96,- The Washington Post Former state House Major...,- The Washington Post,2,0.000967,0.000610,0.997826,0.000598,2
6e5ec6fb,Rubio Comments on Iran Nuclear Deal Editor’s n...,Rubio Comments on Iran Nuclear Deal,1,0.000259,0.998082,0.000260,0.001399,1
d9cd4895,Climate Alarmists Caught Manipulating Temperat...,Climate Alarmists Caught Manipulating Temperat...,1,0.000319,0.996054,0.000708,0.002919,1
4a1a9b9f,Who are the arsonists setting rural fires in W...,Who are the arsonists setting rural fires in W...,1,0.000348,0.993891,0.001137,0.004624,1
6d16fa40,"Diabetes prescriptions now cost NHS £1bn, figu...","Diabetes prescriptions now cost NHS £1bn, figu...",1,0.120679,0.075827,0.740065,0.063430,2
...,...,...,...,...,...,...,...,...
d0b0e459,Global Ocean Circulation Appears To Be Collaps...,Global Ocean Circulation Appears To Be Collaps...,0,0.000255,0.997181,0.000324,0.002239,1
af8069e0,Greenland's ice sheet has melted to a point of...,Greenland's ice sheet has melted to a point of...,2,0.024827,0.012409,0.717568,0.245196,2
3d1a155a,"The Sea Is Rising, but Not Because of Climate ...","The Sea Is Rising, but Not Because of Climate ...",2,0.003819,0.803527,0.003131,0.189523,1


In [10]:
df.to_csv("predictions.tsv", sep="\t")